In [2]:
import pandas as pd

bank_df = pd.read_excel("data/bank_statement.xlsx")
ledger_df = pd.read_excel("data/company_ledger.xlsx")

bank_df.head(), ledger_df.head()


(  Txn_ID    Txn_Date  Amount Currency         Description
 0   B001  2024-01-02    -500      INR       Salary Credit
 1   B002  2024-01-03   -1200      INR        Fuel Expense
 2   B003  2024-01-04    5000      INR     Medical Expense
 3   B004  2024-01-05    5000      INR  Restaurant Payment
 4   B005  2024-01-06   -2500      INR        Fuel Expense,
   Ref_ID    Txn_Date  Amount Currency           Narration
 0   L001  2024-01-03    5000      INR     Mobile Recharge
 1   L002  2024-01-03   -1500      INR  Restaurant Payment
 2   L003  2024-01-04   -1200      INR    Grocery Purchase
 3   L004  2024-01-05   -2000      INR    Electricity Bill
 4   L005  2024-01-06    -500      INR    Grocery Purchase)

In [3]:
# Standardize column names
bank_df = bank_df.rename(columns={
    "Txn_ID": "id",
    "Txn_Date": "date",
    "Amount": "amount",
    "Currency": "currency",
    "Description": "description"
})

ledger_df = ledger_df.rename(columns={
    "Ref_ID": "id",
    "Txn_Date": "date",
    "Amount": "amount",
    "Currency": "currency",
    "Narration": "description"
})

bank_df.head(), ledger_df.head()


(     id        date  amount currency         description
 0  B001  2024-01-02    -500      INR       Salary Credit
 1  B002  2024-01-03   -1200      INR        Fuel Expense
 2  B003  2024-01-04    5000      INR     Medical Expense
 3  B004  2024-01-05    5000      INR  Restaurant Payment
 4  B005  2024-01-06   -2500      INR        Fuel Expense,
      id        date  amount currency         description
 0  L001  2024-01-03    5000      INR     Mobile Recharge
 1  L002  2024-01-03   -1500      INR  Restaurant Payment
 2  L003  2024-01-04   -1200      INR    Grocery Purchase
 3  L004  2024-01-05   -2000      INR    Electricity Bill
 4  L005  2024-01-06    -500      INR    Grocery Purchase)

In [4]:
# Convert date columns to datetime
bank_df["date"] = pd.to_datetime(bank_df["date"])
ledger_df["date"] = pd.to_datetime(ledger_df["date"])

# Merge bank and ledger on amount and date
recon_df = pd.merge(
    bank_df,
    ledger_df,
    on=["date", "amount"],
    how="outer",
    indicator=True,
    suffixes=("_bank", "_ledger")
)

recon_df.head()



,id_bank,date,amount,currency_bank,description_bank,id_ledger,currency_ledger,description_ledger,_merge
0,B001,2024-01-02,-500,INR,Salary Credit,NaN,NaN,NaN,left_only
1,NaN,2024-01-03,-1500,NaN,NaN,L002,INR,Restaurant Payment,right_only
2,B002,2024-01-03,-1200,INR,Fuel Expense,NaN,NaN,NaN,left_only
3,NaN,2024-01-03,5000,NaN,NaN,L001,INR,Mobile Recharge,right_only
4,NaN,2024-01-04,-1200,NaN,NaN,L003,INR,Grocery Purchase,right_only


In [5]:
# Create reconciliation status
recon_df["recon_status"] = recon_df["_merge"].map({
    "both": "Matched",
    "left_only": "Missing in Ledger",
    "right_only": "Missing in Bank"
})

recon_df[["date", "amount", "recon_status"]].head()


,date,amount,recon_status
0,2024-01-02,-500,Missing in Ledger
1,2024-01-03,-1500,Missing in Bank
2,2024-01-03,-1200,Missing in Ledger
3,2024-01-03,5000,Missing in Bank
4,2024-01-04,-1200,Missing in Bank


In [6]:
# Save reconciliation result to Excel
output_path = "output/reconciliation_result.xlsx"
recon_df.to_excel(output_path, index=False)

output_path


'output/reconciliation_result.xlsx'

In [7]:
# Reconciliation summary
summary = recon_df["recon_status"].value_counts().reset_index()
summary.columns = ["Status", "Count"]

summary


,Status,Count
0,Missing in Ledger,28
1,Missing in Bank,25
2,Matched,2


In [10]:
# Save reconciliation result + summary in the same Excel file (different sheets)
with pd.ExcelWriter("output/reconciliation_result.xlsx", engine="openpyxl", mode="w") as writer:
    recon_df.to_excel(writer, sheet_name="Reconciliation_Details", index=False)
    summary.to_excel(writer, sheet_name="Reconciliation_Summary", index=False)

"Summary added to same Excel file"


'Summary added to same Excel file'

In [11]:
# Create a copy for tolerance matching
bank_tol = bank_df.copy()
ledger_tol = ledger_df.copy()

# Rename date columns for tolerance comparison
bank_tol = bank_tol.rename(columns={"date": "bank_date"})
ledger_tol = ledger_tol.rename(columns={"date": "ledger_date"})


In [12]:
# Cross join on amount to allow date tolerance check
date_tol_df = bank_tol.merge(
    ledger_tol,
    on="amount",
    how="inner"
)

# Calculate date difference in days
date_tol_df["date_diff_days"] = (
    date_tol_df["bank_date"] - date_tol_df["ledger_date"]
).abs().dt.days

# Keep only matches within ±1 day
date_tol_matches = date_tol_df[date_tol_df["date_diff_days"] <= 1]

date_tol_matches.head()


,id_x,bank_date,amount,currency_x,description_x,id_y,ledger_date,currency_y,description_y,date_diff_days
7,B002,2024-01-03,-1200,INR,Fuel Expense,L003,2024-01-04,INR,Grocery Purchase,1
8,B003,2024-01-04,5000,INR,Medical Expense,L001,2024-01-03,INR,Mobile Recharge,1
73,B024,2024-01-25,-500,INR,Salary Credit,L023,2024-01-25,INR,Fuel Expense,0
78,B026,2024-01-27,-2500,INR,Grocery Purchase,L024,2024-01-26,INR,Medical Expense,1
79,B026,2024-01-27,-2500,INR,Grocery Purchase,L026,2024-01-27,INR,Electricity Bill,0


In [13]:
# Cross join on date to allow amount tolerance check
amount_tol_df = bank_df.merge(
    ledger_df,
    on="date",
    how="inner",
    suffixes=("_bank", "_ledger")
)

# Calculate amount difference
amount_tol_df["amount_diff"] = (
    amount_tol_df["amount_bank"] - amount_tol_df["amount_ledger"]
).abs()

# Keep only matches within ±5 amount tolerance
amount_tol_matches = amount_tol_df[amount_tol_df["amount_diff"] <= 5]

amount_tol_matches.head()


,id_bank,date,amount_bank,currency_bank,description_bank,id_ledger,amount_ledger,currency_ledger,description_ledger,amount_diff
22,B024,2024-01-25,-500,INR,Salary Credit,L023,-500,INR,Fuel Expense,0
25,B026,2024-01-27,-2500,INR,Grocery Purchase,L026,-2500,INR,Electricity Bill,0
